In [1]:
from iotbx.data_manager import DataManager
from cctbx.array_family import flex
import numpy as np
import tqdm

In [2]:
dm = DataManager()
dm.process_model_file("data/6CVM.cif")
dm.process_real_map_file("data/6cvm/emd_7770.map")
mmm = dm.get_map_model_manager()
assert mmm.map_manager().density_at_sites_cart(mmm.model().get_sites_cart()).as_numpy_array().sum()>0,(
    "Negative or zero density around atoms. Map/model likely not aligned")

In [6]:
from qscore import radial_density_average

In [7]:
# calculate radial profiles v

radial_profiles = [] # radial profile array for each radius r
atoms_all = [] # atoms for each radius r (redundant)
probes_xyz = [] # probe coordinates for each radius r

n_r = 16
rs = np.linspace(0,2,n_r)
for r in tqdm.tqdm(rs):
    profile, probe_xyz, atoms = radial_density_average(mmm,r=r,n=16,selection="all")
    atoms_all.append(atoms)
    radial_profiles.append(profile)
    probes_xyz.append(probe_xyz)

radial_profiles = np.array(radial_profiles).T # to (n_atoms_sel,n_probes)
atoms = atoms_all[0]

# calculate q
mm = mmm.map_manager()
map_values = mm.map_data().as_numpy_array()
Mavg = map_values.mean()
Msigma = map_values.std()
A = Mavg + 10*Msigma
B = Mavg - 1.0*Msigma
u = 0
sigma = 0.6
x = rs
y = A * np.exp(-0.5*((x-u)/sigma)**2) + B
y_flex = flex.double(list(y))

# q for each radial profile
q = np.array([flex.linear_correlation(flex.double(list(x)),y_flex).coefficient() for x in radial_profiles])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:22<00:00,  1.40s/it]
